In [15]:
from dotenv import load_dotenv
load_dotenv()
import os 
from openai import OpenAI 
from pinecone import Pinecone, ServerlessSpec 

In [12]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name='rag', dimension=1536, metric='cosine', spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

In [13]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Smith',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Excellent lecturer, very clear explanations.'},
 {'professor': 'Prof. Johnson',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Brilliant teacher, makes complex concepts easy to understand.'},
 {'professor': 'Dr. Brown',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Knowledgeable but lectures can be a bit dry.'},
 {'professor': 'Prof. Davis',
  'subject': 'History',
  'stars': 4,
  'review': 'Passionate about the subject, engaging lectures.'},
 {'professor': 'Dr. Wilson',
  'subject': 'Chemistry',
  'stars': 5,
  'review': "Best professor I've had, very supportive."},
 {'professor': 'Prof. Martinez',
  'subject': 'Art',
  'stars': 4,
  'review': "Inspires creativity, encourages students' artistic growth."},
 {'professor': 'Dr. Garcia',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Clear and concise teaching style, makes biology interesting.'},
 {'professor': 'Prof. Miller',
  'subje

In [19]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    responce =  client.embeddings.create(
        input=review['review'], 
        model="text-embedding-3-small",
    )
    embedding = responce.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        } 
    })
    

In [20]:
processed_data[0]

{'values': [-0.027814588,
  -0.0052965935,
  -0.022527957,
  -0.014837104,
  -0.012346542,
  -0.010148207,
  -0.030232094,
  0.028983492,
  -0.0136283515,
  0.004287086,
  0.030285226,
  -0.00918519,
  -0.031400997,
  -0.02932885,
  0.028691266,
  0.0026781836,
  0.0068141758,
  0.016311517,
  0.01140345,
  0.052122466,
  0.043727614,
  -0.019406453,
  0.025795573,
  0.041602336,
  -0.05422118,
  -0.074331634,
  0.017626533,
  0.0030019565,
  -0.011310469,
  -0.008242098,
  0.060251657,
  -0.011210847,
  0.01876887,
  -0.03456235,
  -0.04085849,
  0.040831923,
  0.008793342,
  0.0051305564,
  -0.030099263,
  -0.008109268,
  0.018290682,
  0.004087841,
  -0.041867997,
  -0.02879753,
  0.05701061,
  0.013734615,
  -0.04417924,
  -0.0043867086,
  0.03336688,
  0.026114365,
  -0.01903453,
  0.00082728144,
  0.09069628,
  0.032012016,
  -0.050873864,
  0.015620801,
  0.0056917625,
  0.066627495,
  0.01326971,
  -0.046729572,
  0.075288005,
  0.003603012,
  0.039769284,
  0.009118775,
  0.02

In [21]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 21}

In [22]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}